<a href="https://colab.research.google.com/github/usrikanth/Projects/blob/master/sentence_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install transformers


     |████████████████████████████████| 1.8MB 13.4MB/s 
     |████████████████████████████████| 2.9MB 50.4MB/s 
     |████████████████████████████████| 890kB 46.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f0a0b13802731e0df8e66733eae5b61731c7e7375a0f439eb6e1fbd66f9d205a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import warnings
warnings.filterwarnings('ignore')

In [27]:
import re
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()    
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text


In [4]:

#lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords = nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:

lst_stopwords = stopwords.words('english')


In [38]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
text = "We have our preprocessed corpus, consequently the next step is to build the target variable. Basically, we’re here:"
lam = lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True,lst_stopwords=lst_stopwords)
text_clean = lam(text)
print(text_clean)

preprocessed corpus consequently next step build target variable basically


In [22]:
import transformers
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
nlp = transformers.TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [28]:
## tokenize
txt = "river bank"
idx = tokenizer.encode(txt)
print("tokens:", tokenizer.convert_ids_to_tokens(idx))
print("ids   :", tokenizer.encode(txt))
## word embedding
idx = np.array(idx)[None,:]
embedding = nlp(idx)
print("shape:", embedding[0][0].shape)


tokens: ['[CLS]', 'river', 'bank', '[SEP]']
ids   : [101, 2314, 2924, 102]
shape: (4, 768)


In [29]:
## vector of the second input word
embedding[0][0][2]

<tf.Tensor: shape=(768,), dtype=float32, numpy=
array([-7.05829263e-02, -6.21527791e-01, -7.02520549e-01, -2.49586597e-01,
        5.77253520e-01,  4.81010288e-01,  2.44045839e-01,  5.12865007e-01,
       -3.52890432e-01, -8.16163421e-01,  3.99640977e-01,  1.81418151e-01,
        5.12113333e-01,  1.44479528e-01, -1.42232597e+00, -1.06336460e-01,
        1.06119672e-02, -4.34668779e-01,  1.13796592e+00,  7.06453994e-02,
        6.18476391e-01,  5.76436818e-02,  2.41197765e-01,  2.96010047e-01,
       -2.22419769e-01,  2.06983522e-01,  4.50172335e-01,  1.95635021e-01,
        7.22133368e-03,  7.06086218e-01,  6.82602048e-01, -3.35966080e-01,
        4.75640953e-01,  3.38768691e-01, -1.90573201e-01, -1.12145948e+00,
       -4.32029888e-02, -1.56870678e-01,  4.73269597e-02,  3.38264823e-01,
       -4.73710358e-01, -2.87371367e-01, -6.14656471e-02,  3.93012241e-02,
       -9.84509289e-02,  5.61636221e-03,  2.15944827e-01, -3.90441045e-02,
       -7.03861117e-01,  1.26936480e-01,  2.32844744

In [36]:
print(text_clean)

preprocessed corpus consequently next step build target variable basically


In [42]:
## function to apply
def utils_bert_embedding(txt, tokenizer, nlp):
    idx = tokenizer.encode(txt)
    idx = np.array(idx)[None,:]  
    embedding = nlp(idx)
    X = np.array(embedding[0][0][1:-1])
    return X
## create list of news vector
##lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) 
#                 for txt in dtf["text_clean"]]
## create the feature matrix (n news x 768)
#X = np.array(lst_mean_vecs)
mean_vec = utils_bert_embedding(text_clean, tokenizer, nlp).mean(0)

In [43]:
mean_vec

array([-2.97941893e-01, -3.24873954e-01,  2.71756649e-01, -1.92078099e-01,
        3.37075233e-01,  7.63574010e-03,  2.96800584e-01,  4.13517922e-01,
        1.25400215e-01, -3.27635556e-01, -1.02113359e-01, -1.19906552e-01,
       -2.28428006e-01,  3.24692816e-01, -3.62101883e-01,  1.54968500e-01,
       -5.77161200e-02, -1.30803669e-02,  3.33321333e-01,  8.60894565e-03,
        2.08776370e-01, -3.31259578e-01, -6.44524813e-01,  5.75353019e-02,
        7.35732138e-01, -4.23924178e-01, -1.68876782e-01, -3.24280500e-01,
       -7.42046893e-01,  5.74545749e-02, -3.62124473e-01,  3.88969153e-01,
        6.52799085e-02, -4.25466001e-02, -5.05130226e-03,  3.13918859e-01,
        1.81166932e-01,  3.20932120e-01, -2.66320705e-01,  6.90403059e-02,
       -3.74672771e-01, -4.71886069e-01,  1.07147349e-02, -7.51571059e-02,
       -4.49765950e-01, -3.08078378e-01,  1.71110153e-01,  1.78092822e-01,
       -9.05257389e-02, -5.08079678e-02, -9.24688816e-01,  2.77273208e-01,
       -2.51149237e-01,  